# Instructions to run the Program

#Run block by block and the output can be observed below the respective cell.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
from sklearn import linear_model
import seaborn as sn


# Data Handling

In [2]:

x_train = pd.read_table('C:/Users/rakshith/Downloads/UCI/train/X_train.txt', delim_whitespace=True,header=None)
x_test = pd.read_csv('C:/Users/rakshith/Downloads/UCI/test/x_test.txt',delim_whitespace=True,header=None)
y_train = pd.read_table('C:/Users/rakshith/Downloads/UCI/train/y_train.txt',delim_whitespace=True,header=None)
y_test = pd.read_csv('C:/Users/rakshith/Downloads/UCI/test/y_test.txt',delim_whitespace=True,header=None)

dickey = {"1":"Walking", "2":"Moving_Upstairs","3":"Moving_Downstairs","4":"Sitting","5":"Standing","6":"Laying"}


C:\Users\rakshith\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  
C:\Users\rakshith\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  after removing the cwd from sys.path.


# Normalization

In [3]:
x_train_mean = x_train.mean(axis=0)
x_test_mean = x_test.mean(axis=0)
x_train_sd = x_train.std(axis = 0)
x_test_sd = x_test.std(axis = 0)

x_train = (x_train - x_train_mean)/x_train_sd  
x_test = (x_test - x_test_mean)/x_test_sd 
y_train = y_train              
y_test = y_test               
y = np.array(y_train).ravel() 

#Copying data for other models
x_train_common = x_train
y_train_common = y_train
x_test_common = x_test
y_test_common = y_test


# Logisitic Regression

In [4]:
#Logistic regression

import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

#                                                       ****** Training part  *********
logistic = linear_model.LogisticRegression()
logistic.fit(x_train, y_train)
train_accuracy = logistic.score(x_train, y_train)


#                                             ********  Validation and Hyperparameter tuning  *********                                  
C = [0.001, 0.1,0.5, 0.9, 1, 10]
penalty = ['l1', 'l2']
hyper = dict(penalty=penalty, C=C)

  #5-fold cross validation and tuning
logistic_validated = GridSearchCV(estimator=logistic, param_grid=hyper, cv =5) 

validation_result = logistic_validated.fit(x_train, y_train)

print("Validation Accuracy:", validation_result.best_score_*100)
print('Best C:', validation_result.best_estimator_.get_params()['C'])
print('Best Penalty:', validation_result.best_estimator_.get_params()['penalty'])


#                                              **********       Testing     ************
prediction_result = validation_result.predict(x_test)
print("Testing Accuracy:", accuracy_score(y_test, prediction_result)*100)
print()


#                                              ***********   Confusion Matrix   **********
predicted_y = list(map(str, prediction_result))
y_test_data = np.array(y_test.T).ravel()
original_y = list(map(str,y_test_data))
pred_y1 = np.array(pd.Series(predicted_y).map(dickey))
y_test_key = np.array(pd.Series(original_y).map(dickey))

array = confusion_matrix(pred_y1, y_test_key)
logistic_cmatrix = pd.DataFrame(array, range(6),range(6))
logistic_cmatrix.columns = ["Laying", "Sitting","Standing","Walking","Walk_downstairs","W_Upstairs"]
logistic_cmatrix.index = ["Laying", "Sitting","Standing","Walking","W_downstairs","W_Upstairs"]

#                                              ***********     Confusion Matrix table *******
print(logistic_cmatrix)
print()
activities = ["Laying", "Sitting","Standing","Walking","Walking_downstairs","Walking_Upstairs"]
classification_report(y_test, prediction_result)
print("Classification Analysis:")
print(classification_report(y_test, prediction_result, target_names=activities))


Validation Accuracy: 94.19205658324266
Best C: 0.9
Best Penalty: l1
Testing Accuracy: 96.50492025788938

              Laying  Sitting  Standing  Walking  Walk_downstairs  W_Upstairs
Laying           537        0         0        0                0           0
Sitting            0      413         1        0                0           3
Standing           0        6       464        2                0           6
Walking            0        0         0      427               16           0
W_downstairs       0        0         0       62              516           0
W_Upstairs         0        1         6        0                0         487

Classification Analysis:
                    precision    recall  f1-score   support

            Laying       0.99      0.98      0.98       496
           Sitting       0.97      0.99      0.98       471
          Standing       0.99      0.98      0.99       420
           Walking       0.96      0.87      0.91       491
Walking_downstairs    

# 

# SVM

In [5]:
x_train = x_train_common
x_test = x_test_common
y_train = y_train_common
y_test = y_test_common

In [6]:
#Support Vector Classification

import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

#                                                       ****** Training part  *********
svc = SVC(kernel='rbf')
svc.fit(x_train, y_train) 
train_accuracy = svc.score(x_train, y_train)


#                                             ********  Validation and Hyperparameter tuning  *********           
C = [0.01, 0.1, 1]#[0.01, 0.1, 1, 10]
gammas = [0.01, 0.1]

param_grid = {'C': C, 'gamma' : gammas }
svc_validated = GridSearchCV(svc, param_grid, cv=5)
validation_result = svc_validated.fit(x_train, y_train)

print("Validation Accuracy:", validation_result.best_score_*100)
print('Best C:', validation_result.best_estimator_.get_params()['C'])
print('Best Gamma:', validation_result.best_estimator_.get_params()['gamma'])


#                                              **********       Testing     ************
prediction_result = validation_result.predict(x_test)
print("Testing Accuracy:", accuracy_score(y_test, prediction_result)*100)
print()


#                                              ***********   Confusion Matrix   **********
predicted_y = list(map(str, prediction_result))
y_test_data = np.array(y_test.T).ravel()
original_y = list(map(str,y_test_data))
pred_y1 = np.array(pd.Series(predicted_y).map(dickey))
y_test_key = np.array(pd.Series(original_y).map(dickey))

array = confusion_matrix(pred_y1, y_test_key)
logistic_cmatrix = pd.DataFrame(array, range(6),range(6))
logistic_cmatrix.columns = ["Laying", "Sitting","Standing","Walking","Walk_downstairs","W_Upstairs"]
logistic_cmatrix.index = ["Laying", "Sitting","Standing","Walking","W_downstairs","W_Upstairs"]

#                                              ***********     Confusion Matrix table *******
print(logistic_cmatrix)
print()
activities = ["Laying", "Sitting","Standing","Walking","Walking_downstairs","Walking_Upstairs"]
classification_report(y_test, prediction_result)
print("Classification Analysis:")
print(classification_report(y_test, prediction_result, target_names=activities))


Validation Accuracy: 84.52121871599564
Best C: 1
Best Gamma: 0.01
Testing Accuracy: 86.08754665761792

              Laying  Sitting  Standing  Walking  Walk_downstairs  W_Upstairs
Laying           499        0         0        1                0           0
Sitting           38      392        96       24               24         131
Standing           0       26       368        0                0           0
Walking            0        0         0      434               29           0
W_downstairs       0        0         0       32              479           0
W_Upstairs         0        2         7        0                0         365

Classification Analysis:
                    precision    recall  f1-score   support

            Laying       0.98      0.74      0.84       496
           Sitting       0.93      0.78      0.85       471
          Standing       0.56      0.93      0.70       420
           Walking       0.94      0.88      0.91       491
Walking_downstairs      

# Neural Nets 

In [7]:
x_train = x_train_common
x_test = x_test_common
y_train = y_train_common
y_test = y_test_common

In [8]:
#Neural nets
import warnings
warnings.filterwarnings('ignore')

from sklearn.neural_network import MLPClassifier

#                                                       ****** Training part  *********
neural_net = MLPClassifier(alpha=5,hidden_layer_sizes=(10, 6), random_state=1,solver='adam')
neural_net.fit(x_train, y_train) 
#train_accuracy = neural_net.score(x_train, y_train)


#                                             ********  Validation and Hyperparameter tuning  *********    
hyper = {'max_iter': [10,50,100], 'hidden_layer_sizes':np.arange(5, 10), 'alpha': [0.001, 0.01, 0.1]}  # 'alpha': 10.0 ** -np.arange(1, 7), '

nn_validated = GridSearchCV(neural_net, param_grid=hyper, cv=5)
validation_result = nn_validated.fit(x_train, y_train)



In [9]:
print("Validation Accuracy:", validation_result.best_score_*100)

print("Best Values for Hypeparameters:")
print(validation_result.best_params_)


#                                              **********       Testing     ************
prediction_result = validation_result.predict(x_test)
print("Testing Accuracy:", accuracy_score(y_test, prediction_result)*100)
print()


#                                              ***********   Confusion Matrix   **********
predicted_y = list(map(str, prediction_result))
y_test_data = np.array(y_test.T).ravel()
original_y = list(map(str,y_test_data))
pred_y1 = np.array(pd.Series(predicted_y).map(dickey))
y_test_key = np.array(pd.Series(original_y).map(dickey))

array = confusion_matrix(pred_y1, y_test_key)
logistic_cmatrix = pd.DataFrame(array, range(6),range(6))
logistic_cmatrix.columns = ["Laying", "Sitting","Standing","Walking","Walk_downstairs","W_Upstairs"]
logistic_cmatrix.index = ["Laying", "Sitting","Standing","Walking","W_downstairs","W_Upstairs"]

#                                              ***********     Confusion Matrix table *******
print(logistic_cmatrix)
print()
activities = ["Laying", "Sitting","Standing","Walking","Walking_downstairs","Walking_Upstairs"]
classification_report(y_test, prediction_result)
print("Classification Analysis:")
print(classification_report(y_test, prediction_result, target_names=activities))


Validation Accuracy: 92.88628944504896
Best Values for Hypeparameters:
{'alpha': 0.001, 'hidden_layer_sizes': 9, 'max_iter': 100}
Testing Accuracy: 93.92602646759416

              Laying  Sitting  Standing  Walking  Walk_downstairs  W_Upstairs
Laying           520        0         0        2                0           0
Sitting            0      398         6        0                0          11
Standing           0       21       445        1                0          11
Walking            2        0         0      465               66           0
W_downstairs      15        0         0       23              466           0
W_Upstairs         0        1        20        0                0         474

Classification Analysis:
                    precision    recall  f1-score   support

            Laying       0.96      0.96      0.96       496
           Sitting       0.93      0.94      0.94       471
          Standing       0.96      0.95      0.95       420
           Walking  